In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

from noteify.core.config import SAMPLE_RATE
from noteify.core.datasets import (MusicNetDataset, MusicNetDatasetProcessed, MusicAugmentor,
                                   MusicNetSampler, get_musicnet_dataloader)
from noteify.core.training import make_optimizer, make_scheduler, train_model
from noteify.core.models import TranscriptionNN
from noteify.core.utils import plot_audio, plot_roll_info
from noteify.utils import get_rel_pkg_path

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("Warning: Could not find GPU! Using CPU only")

Using the GPU!


In [3]:
data_dir = get_rel_pkg_path("musicnet/")
save_dir = get_rel_pkg_path("weights/")

In [4]:
raw_dataset_train = MusicNetDataset(data_dir, download=True, train=True,
                                    numpy_cache=True, piano_only=True) #, filter_records=[1727])
raw_dataset_test = MusicNetDataset(data_dir, download=True, train=False,
                                   numpy_cache=True, piano_only=True) #, filter_records=[1759])

  2%|█▉                                                                              | 15/640 [00:00<00:04, 134.82it/s]

Loading audio


100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 1571.28it/s]

Loading audio


In [5]:
batch_size = 32

In [6]:
dataset_train = MusicNetDatasetProcessed(raw_dataset_train, augmentor=MusicAugmentor())
sampler_train = MusicNetSampler(dataset_train, batch_size, random_start_times=True)
workers = 3 if raw_dataset_train.piano_only else 2
dataloader_train = get_musicnet_dataloader(dataset_train, sampler_train, num_workers=workers, pin_memory=True)

dataset_test = MusicNetDatasetProcessed(raw_dataset_test)
sampler_test = MusicNetSampler(dataset_test, batch_size)
dataloader_test = get_musicnet_dataloader(dataset_test, sampler_test, num_workers=None, pin_memory=True)

dataloaders = {'train': dataloader_train, 'test': dataloader_test}

In [7]:
model = TranscriptionNN()
model = model.to(device)
past_train_weights = os.path.abspath(os.path.join(save_dir, "Experiment 12-07-2020 08-09PM/Weights Best.pckl"))
model.load_state_dict(torch.load(past_train_weights))

CQT kernels created, time used = 0.0956 seconds


<All keys matched successfully>

In [8]:
optimizer = make_optimizer(model, lr=0.001)

In [9]:
num_epochs = 50
scheduler = make_scheduler(optimizer, [60], gamma=0.5)

In [ ]:
tracker = train_model(device=device,
                      model=model,
                      dataloaders=dataloaders,
                      optimizer=optimizer,
                      lr_scheduler=scheduler,
                      save_model=True,
                      save_dir=save_dir,
                      save_best=True,
                      save_all=False,
                      num_epochs=num_epochs)

  0%|                                                                                          | 0/307 [00:00<?, ?it/s]

Epoch 0/49
----------


Avg. Loss: 0.2092, Total Loss: 0.1910, Loss Parts: [0.1313, 0.0297, 0.0300]:  15%|▌   | 46/307 [02:55<04:37,  1.06s/it]

In [ ]:
%qtconsole